In [30]:
#general imports
import numpy as np
#import autograd.numpy as np #to support SGD implementation
import matplotlib.pyplot as plt
import time
import pandas as pd

MNIST Data

In [31]:
train_path = '/home/wpm/Documents/UW/Spring_2023/AMATH_563/HW/HW2/code/MNIST/mnist_train.csv'
test_path = '/home/wpm/Documents/UW/Spring_2023/AMATH_563/HW/HW2/code/MNIST/mnist_test.csv'

Kernel MMD Generator

In [32]:
#Imports for MMD model
from scipy.optimize import minimize

In [33]:
#Data Processing
def csv_to_matrix(path: str):
    '''
    Turns csv filepath string of Hw2 data into X,y arrays
    '''
    df = pd.read_csv(path)
    #X = df.iloc[:,1:] #all rows
    #y = df.iloc[:,0] #all rows
    X = df.iloc[:100,1:] #small sample
    y = df.iloc[:100,0] #small sample
    return np.array(X), np.array(y)

In [34]:
#Kernels
def generate_kern_matrix(X, Y, kern_func):
    '''
    Creates K(X,Y) for specific kernel function ("Y" is a data matrix, not labels)
    '''
    n = X.shape[0]
    m = Y.shape[0]
    K = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            K[i, j] = kern_func(X[i], Y[j])
    return K

def gaussian_kernel(x, y, sigma=1.0):
    '''
    Gaussian kernel function (defined on vectors; can be used for both scalar and operator kernels)
    '''
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * sigma**2))

In [35]:
#Loss
def kern_sum(X1,X2,kern_func):
    '''
    Helper func for mmd
    '''
    n = X1.shape[0]
    m = X2.shape[0]
    return (1/n)*(1/m)*sum( [sum( [kern_func(X1[i],X2[j]) for i in range(n)]) for j in range(m)] )

def mmd(X1,X2,kern_func):
    '''
    Scalar kernel MMD portion of the loss
    '''
    X1_norm = kern_sum(X1,X1,kern_func)
    X2_norm = kern_sum(X2,X2,kern_func)
    cross_norm = kern_sum(X1,X2,kern_func)
    return X1_norm+X2_norm-2*cross_norm

def regularizer(lam,Z,Q_inv):
    '''
    The operator-kernel regularizer
    '''
    d = Z.shape[1]
    return lam * sum( [ Z[:,j]@Q_inv@Z[:,j].T for j in range(d) ] )

def loss(Z_flat,y,kern_func,lam,Q_inv,shape):
    '''
    Precise loss (no autograd/SGD)
    '''
    Z = Z_flat.reshape(shape)
    return mmd(Z,y,kern_func) + regularizer(lam,Z,Q_inv)

In [36]:
def sample_generator(Xte,Xtr,Q_inv,Z,d):
    '''
    Generate samples from latent space (send them through the map)
    '''
    Q_test = generate_kern_matrix(Xte,Xtr,gaussian_kernel)
    T = [Q_test @ Q_inv @ Z[:,j].T for j in range(d)]
    return np.array(T) #might need to be transposed

In [37]:
from sklearn.decomposition import PCA
#Implementation
LAM=0.01 #lambda for regularizer

#Datasets
Xtr,ytr = csv_to_matrix(train_path) #trainset
Xte,yte = csv_to_matrix(test_path) #testset

#Dim redux
pca = PCA(n_components=10)
Xtr = pca.fit_transform(Xtr)
Xte = pca.transform(Xte)
n,d = Xtr.shape #cached dimensions

#Op kernel (Gaussian)
print('Generating kernel matrix')
Q = generate_kern_matrix(Xtr, Xtr, gaussian_kernel)
print('Inverting said matrix')
Q_inv = np.linalg.inv(Q)
print('K(X,X) inverted')

Generating kernel matrix
Inverting said matrix
K(X,X) inverted


In [39]:
#Z
print('Optimizing Z') #takes long-ass time
X0 = np.ones((n,d))
X0_flat = X0.flatten()
Z_argmin_flat = minimize(loss, X0_flat, args=(ytr, gaussian_kernel, LAM, Q_inv, (n,d))) #MMD kernel also Guassian
Z_star = Z_argmin_flat.x.reshape((n,d))

Optimizing Z


KeyboardInterrupt: 

In [ ]:
#The map
print('Generating samples')
T = sample_generator(Xte,Xtr,Q_inv,Z_star,d)

print(T.shape)
print(T)

AttributeError: 'PCA' object has no attribute 'shape'